# Sklearn Model Inference

https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/scikit_learn_iris/scikit_learn_estimator_example_with_batch_transform.ipynb

In [2]:
import boto3
import numpy as np
import pandas as pd
import os
import sagemaker
from sagemaker import get_execution_role
#from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
# S3 prefix
bucket = "mobi.source.data.batch.dev"
prefix = "dedup"
category = "lodging"
version = "version_1707151509"

FRAMEWORK_VERSION = "1.2-1"

#train_input = f's3://{bucket}/{prefix}/sagemaker/input_data/train.csv'

batch_input_s3 = f's3://{bucket}/{prefix}/addfeatures/{category}/new/{version}'
#batch_input_s3 = f's3://{bucket}/{prefix}/debug/addfeatures/{category}/1699806821/run-1703787890584-part-r-00019'
batch_output_s3 = f's3://{bucket}/{prefix}/inference/{category}/new/{version}'
prefix_to_delete = f'{prefix}/inference/{category}/new/{version}'  # for clean up output folders

## Batch Transform

In [4]:
# clean up batch_output_s3
s3_client = boto3.client('s3')

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix_to_delete)

if response.get('Contents'):
    for object in response['Contents']:
        s3_client.delete_object(Bucket=bucket, Key=object['Key'])
        # print('Deleting', object['Key'])

In [5]:
# load sklearn model
model_package_group_name = "dedup-model-package-group-gradient"
sagemaker_boto_client = sagemaker_session.boto_session.client("sagemaker")
response = sagemaker_boto_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    MaxResults=1,
    SortBy="CreationTime",
    SortOrder="Descending",
    ModelApprovalStatus="Approved",
)

latest_model_arn = response["ModelPackageSummaryList"][0]["ModelPackageArn"]
print(latest_model_arn)


arn:aws:sagemaker:us-east-1:769026081114:model-package/dedup-model-package-group-gradient/1


In [6]:
from sagemaker import ModelPackage

sklearn = ModelPackage(role=role, model_package_arn=latest_model_arn)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
# Define an SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn.transformer(instance_count=4, 
                                  instance_type="ml.m5.xlarge", 
                                  max_payload=20,
                                  max_concurrent_transforms=4,
                                  output_path=batch_output_s3,
                               #   strategy="MultiRecord", 
                                  accept="text/csv", 
                                  assemble_with="Line")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Check Output Data

In [9]:
transformer.output_path

's3://mobi.source.data.batch.dev/dedup/inference/lodging/new/version_1707151509'